In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaymann/car-number-plate-dataset-yolo-format")

print("Path to dataset files:", path)

c:\Users\wafat\anaconda3\envs\VC_P4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 203M/203M [00:15<00:00, 13.6MB/s] 

Extracting files...


Path to dataset files: C:\Users\wafat\.cache\kagglehub\datasets\sujaymann\car-number-plate-dataset-yolo-format\versions\3


In [2]:
import torch
from ultralytics import YOLO
import cv2
import csv
import time

# Configuración mínima
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolo11n.pt')
video_path = "vc_coches.mp4"

# Intentar importar EasyOCR, si no está, usar un placeholder
try:
    import easyocr
    easy_reader = easyocr.Reader(['en'])
    OCR_AVAILABLE = True
    print("EasyOCR disponible - procesando matrículas")
except ImportError:
    OCR_AVAILABLE = False
    print("EasyOCR no disponible - solo detección de objetos")

# Abrir video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: No se puede abrir el video")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("salida_simple.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# CSV simple
csv_file = open("detecciones_simple.csv", 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame", "objeto", "confianza", "id", "matricula"])

frame_id = 0
tiempos_ocr = []

def leer_matricula_simple(imagen):
    """Función simplificada para leer matrículas"""
    if not OCR_AVAILABLE or imagen.size == 0:
        return ""
    
    start = time.time()
    try:
        # Redimensionar para mejor OCR
        if imagen.shape[1] < 100:
            escala = 100 / imagen.shape[1]
            nueva_w = int(imagen.shape[1] * escala)
            nueva_h = int(imagen.shape[0] * escala)
            imagen = cv2.resize(imagen, (nueva_w, nueva_h))
        
        # Convertir a grises y mejorar contraste
        gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
        gris = cv2.equalizeHist(gris)
        
        # Usar EasyOCR
        resultados = easy_reader.readtext(gris, detail=1, paragraph=False)
        
        if resultados:
            # Tomar el resultado con mayor confianza
            mejor = max(resultados, key=lambda x: x[2])
            texto = ''.join(filter(lambda c: c.isalnum() or c in '- ', mejor[1])).upper()
            
            # Filtrar textos muy cortos o poco probables
            if len(texto) >= 4 and any(c.isdigit() for c in texto):
                tiempos_ocr.append(time.time() - start)
                return texto
            
    except Exception as e:
        print(f"Error en OCR: {e}")
    
    tiempos_ocr.append(time.time() - start)
    return ""

print("Procesando video...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_id += 1
    if frame_id % 30 == 0:  # Mostrar progreso cada ~1 segundo
        print(f"Frame {frame_id}...")

    # Detectar solo coches (clase 2)
    results = model.track(frame, persist=True, classes=[2], device=device, verbose=False)

    if results and results[0].boxes is not None:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            track_id = int(box.id[0]) if box.id is not None else -1

            matricula = ""
            
            # Solo intentar OCR si está disponible
            if OCR_AVAILABLE:
                # Buscar matrícula en la parte inferior del coche
                altura_coche = y2 - y1
                region_h = max(30, altura_coche // 4)  # 25% de la altura del coche
                region_matricula = frame[y2-region_h:y2, x1:x2]
                
                if region_matricula.size > 0:
                    matricula = leer_matricula_simple(region_matricula)

            # Dibujar bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Texto con matrícula si se detectó
            texto = f"Car {track_id}"
            if matricula:
                texto += f" - {matricula}"
                cv2.putText(frame, matricula, (x1, y2 + 25), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            cv2.putText(frame, texto, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Guardar en CSV
            csv_writer.writerow([frame_id, "car", f"{conf:.2f}", track_id, matricula])

    out.write(frame)
    
    # Mostrar preview
    cv2.imshow("Deteccion Vehiculos", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
        break

# Limpieza
cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()

print(f"\nProcesados {frame_id} frames")
if tiempos_ocr:
    print(f"Tiempo promedio OCR: {sum(tiempos_ocr)/len(tiempos_ocr)*1000:.1f}ms")
    print(f"Total lecturas OCR: {len(tiempos_ocr)}")

EasyOCR no disponible - solo detección de objetos
Procesando video...

Procesados 15 frames
